# INPUT

La première étape consiste en une observation, nous avons un dossier nommé "data" contenant 11 fichiers csv.
Nous créons ce notebook à l'intérieur de ce dossier pour appeler les 11 csv sans path.

La deuxième étape consiste à créer la liste des noms de chacun des dataframes afin de pouvoir ensuite les appeler et les stocker dans un dictionnaire.

La troisième étape consiste à mettre les 11 dataframes dans le dictionnaire en couplant les noms des exchanges en KEY avec les dataframes correspondants en VALUE pour qu'il restent attachés l'un a l'autre pendant tout le processus de calcul de moyenne pondérée (VWAP), d'écart-type (STD) et d'OHLC.

## LISTES DES VARIABLES

Dans un but documentaire, et pour faciliter la lecture du code, voici une liste de variables et d'objets créés dans ce notebook.

### 1 liste
liste_noms = liste contenant les 11 noms d'exchange


### 1 séries

amount_global_serie = série contenant les 11 volumes cumulés


### 5 dictionnaires
dict_init = dictionnaire contenant les 11 dataframes initiaux 

dict_dim = dictionnaire contenant les dimensions des 11 dataframes

dict_vwap = dictionnaire contenant le VWAP resamplés des 11 dataframes

dict_amount = dictionnaire contenant les volumes resamplés des 11 dataframes

dict_amount_cum = dictionnaire contenant les 11 volumes cumulés (sommes de chaque ligne)

### 5 Dataframes
dim_df = dataframe pour présenter proprement les dimensions des 11 dataframes

vwap_df = dataframe contenant les VWAP en 11 colonnes + une colonne de VWAP global

amount_df = dataframe contenant les volumes resamplés en 11 colonnes

std_df = dataframe contenant les STD en 11 colonnes

ohlc_df = dataframe concaténé verticalement contenant les OHLC en 11x4 colonnes


### 2 fonctions customs

calcul_vwap = fonction de calcul de VWAP

calcul_sum_amount = fonction de calcul de volume cumulé

### CHOIX DE L'ECHANTILLON PAR L'UTILISATEUR

Cet input vous laisse le choix de l'échantillon temporel de découpage des tranches horaires étudiées. 
Entrez un entier comme durée en minutes de l'échantillon temporel et compilez tout le notebook. 
Le programme vous sortira les dataframes de moyenne pondérée, d'écart-type, et d'OHLC pour chaque tranche. 
Il les enregistrera aussi automatiquement en csv dans le dossier output_data. 
Pour recommencer, redemarrez le noyau et choisissez un autre échantillon. 

In [1]:
# Veuillez entrer une durée en minutes
while True:
    time_input = int(input("Échantillon en minutes: "))

    if time_input <= 0:
        print(f"Votre choix: {time_input} n'est pas un entier positif. Merci de recommencer")
    else:
        time_sample = f"{str(time_input)}min"
        print(time_sample)
        break 

Échantillon en minutes:  20


20min


In [2]:
#import des librairies utiles au projet:
import numpy as np
import pandas as pd
import os

In [4]:
# Création d'une liste vide pour y mettre les 11 noms d'exchanges sans l'extansion '.csv':
liste_noms = []

# Chemin vers le dossier input_data
input_data_path = 'input_data'

# Cette fonction coupe le nom du fichier csv au niveau du point et ne garde que la premiere partie
for filename in os.listdir(input_data_path):
    # os ne prend que les fichiers avec une extension '.csv'
    if os.path.splitext(filename)[1] == '.csv':
        
         # OS ne prend que la premiere partie du nom sans l'extensions '.csv'
        file = os.path.splitext(filename)[0]
        
        # La liste est remplie par itération
        liste_noms.append(file)
        
# Les 11 noms sont classés par ordre alphabétique        
liste_noms.sort()

liste_noms

['bfly',
 'bfnx',
 'bnus',
 'btrx',
 'cbse',
 'gmni',
 'itbi',
 'krkn',
 'lmax',
 'okcn',
 'stmp']

Ce procédé peut sembler long pour écrire 11 strings de 4 lettres, mais il devient très utile si la liste des csv se rallonge. Et surtout, il evite les erreurs de saisie et les oublis.

In [5]:
# Création d'un dictionnaire vide
dict_init = {}

# Lire chaque fichier CSV les intégrer dans le dictionnnaire:
for nom in liste_noms:
    
    file_path = os.path.join(input_data_path, nom + '.csv')
    
    # Cette boucle for associe chaque dataframe créé avec son nom dans une relation KEY:VALUE
    dict_init[nom] = pd.read_csv(file_path, sep=',')

dict_init

{'bfly':                    timestamp     price    amount
 0    2021-02-24 23:59:33.720  49712.15  0.005400
 1    2021-02-24 23:44:16.980  49603.44  0.077000
 2    2021-02-24 22:32:39.320  48864.19  0.120000
 3    2021-02-24 22:15:19.030  48252.47  0.120000
 4    2021-02-24 22:15:17.157  48247.38  0.120000
 ..                       ...       ...       ...
 229  2021-02-24 00:14:48.747  48263.33  0.250300
 230  2021-02-24 00:11:03.583  48574.05  0.000245
 231  2021-02-24 00:11:02.263  48574.05  0.001000
 232  2021-02-24 00:05:08.513  48683.05  0.010000
 233  2021-02-24 00:02:59.720  48929.54  0.120000
 
 [234 rows x 3 columns],
 'bfnx':                       timestamp         price    amount
 0       2021-02-24 23:59:58.181  49716.000000  0.010000
 1       2021-02-24 23:59:58.176  49713.000000  0.005000
 2       2021-02-24 23:59:51.526  49715.000000  0.005000
 3       2021-02-24 23:59:46.698  49726.000000  0.005000
 4       2021-02-24 23:59:46.353  49715.000000  0.005000
 ...           

In [6]:
# Observation du nombre de lignes et de colonnes de chaque table

# Création d'un dictionnaire vide
dict_dim = {}

# La boucle for joint en couple les noms des exchanges et le nombre de ventes
for nom, df in dict_init.items():
    dict_dim[nom] = df.shape
    
# Nous le transformons en Dataframe pour le présenter plus proprement
index = ('lignes', 'colonnes')
dim_df = pd.DataFrame(dict_dim, index=index)
dim_df

,bfly,bfnx,bnus,btrx,cbse,gmni,itbi,krkn,lmax,okcn,stmp
lignes,234,318667,53672,26789,489110,49329,30187,75090,27972,12459,53284
colonnes,3,3,3,3,3,3,3,3,3,3,3


# MANIPULATION DES DONNEES ET CALCUL

Nous constatons que tous les dataframes ont des dimensions verticales  (nombre de lignes) tres différentes (de 234 à 489110 lignes) mais la même structure en 3 colonnes, qui sont: timestamp, price et amount.
Chaque ligne correspond à une vente de bitcoin avec les 3 informations essentielles, date, prix, volume. Les dates sont très précises jusqu'au millième de seconde, sauf pour le dernier exchange STMP, qui lui se limite à la seconde.

Maintenant que les données sont importées, nous disposons des noms des variables, de la liste des 11 noms et du dictionnaire contenant les 11 dataframes, et que nous connaissons leur dimensions nous allons les manipuler.

### Les étapes seront les suivantes.

1) Changement de la colonne timestamp en index au format DateTime
2) Resampling des dataframes en calculant le prix moyen pondéré par le volume de chaque tranche d'heure "resamplée"
3) Interpolation des valeurs manquantes
4) Création d'un dataframe regroupant les 11 VWAP en 11 colonnes
5) Calcul du volume cumulé de chaque exchange
6) Création d'une 12eme colonne dans le dataframe final pour y placer le calcul de la moyenne pondérée globale
7) Export du dataframe en format '.csv' vers un dossier outdata

In [7]:
# Transformation de colonne timestamp en DateTimeIndex
for df in dict_init.values():
    # Cette fonction permet de caster les dates en datetime pour les uniformiser
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    
    # Cette fonction transforme la colonne timestamp en index des dataframes
    df = df.set_index('timestamp',  inplace=True)
dict_init

{'bfly':                             price    amount
 timestamp                                  
 2021-02-24 23:59:33.720  49712.15  0.005400
 2021-02-24 23:44:16.980  49603.44  0.077000
 2021-02-24 22:32:39.320  48864.19  0.120000
 2021-02-24 22:15:19.030  48252.47  0.120000
 2021-02-24 22:15:17.157  48247.38  0.120000
 ...                           ...       ...
 2021-02-24 00:14:48.747  48263.33  0.250300
 2021-02-24 00:11:03.583  48574.05  0.000245
 2021-02-24 00:11:02.263  48574.05  0.001000
 2021-02-24 00:05:08.513  48683.05  0.010000
 2021-02-24 00:02:59.720  48929.54  0.120000
 
 [234 rows x 2 columns],
 'bfnx':                                 price    amount
 timestamp                                      
 2021-02-24 23:59:58.181  49716.000000  0.010000
 2021-02-24 23:59:58.176  49713.000000  0.005000
 2021-02-24 23:59:51.526  49715.000000  0.005000
 2021-02-24 23:59:46.698  49726.000000  0.005000
 2021-02-24 23:59:46.353  49715.000000  0.005000
 ...                         

In [8]:
# Création de la fonction de calcul de la moyenne des prix pondérée par les volumes
def calcul_vwap(x):
    # np.nan évite une possible division par zéro
    x = np.nan if x['amount'].sum() == 0 else ((x['price'] * x['amount']).sum() / x['amount'].sum())
    return x

# Création de la fonction de calcul de somme cumulée des volumes
def calcul_sum_amount(x):
    x = np.nan if x['amount'].sum() == 0 else x['amount'].sum()
    return x

In [9]:
# Création d'un nouveau dictionnaire contenant les prix moyens pondérés par le volume
# pour chaque heure, en appelant la fonction de calcule de moyenne pondérée
dict_vwap = {}
for k, v in dict_init.items():
    dict_vwap[k] = v.resample(time_sample).apply(calcul_vwap)
#dict_vwap

In [10]:
# Création d'un nouveau dictionnaire contenant le cumul des volumes pour chaque heure, 
# en appelant la fonction de calcule de volume cumulé.
dict_amount = {}
for k, v in dict_init.items():
    dict_amount[k] = v.resample(time_sample).apply(calcul_sum_amount)    
#dict_amount

In [11]:
# Nous constatons la présence d'un NaN à 5h du matin dans l'exchange bfly
# Comme un prix évolue de façon linéaire entre deux points, nous choisissons de le combler par interpolation
for nom in liste_noms:
    dict_vwap[nom].interpolate(inplace=True)
    dict_amount[nom].interpolate(inplace=True)

In [12]:
# Création des 2 dataframes de dimension 24 lignes x 11 colonnes à partir des dictionnaires
vwap_df = pd.DataFrame(dict_vwap)
amount_df = pd.DataFrame(dict_amount)

In [13]:
# Dataframe contenant les VWAP des 11 exchanges
vwap_df.head()

,bfly,bfnx,bnus,btrx,cbse,gmni,itbi,krkn,lmax,okcn,stmp
timestamp,,,,,,,,,,,
2021-02-24 00:00:00,48249.222150,48258.479492,48332.214856,48362.278289,48395.779288,48634.707736,48320.808471,48167.026516,48457.997296,48094.876170,48472.607232
2021-02-24 00:20:00,47444.868821,47362.339719,47456.668198,47357.619664,47408.711379,47377.433422,47359.097461,47339.089475,47344.067631,47362.588674,47309.322432
2021-02-24 00:40:00,48129.147408,47976.235747,47945.813147,48037.306405,47989.239927,47835.837186,47993.418746,48054.618736,47900.208287,48018.952629,47930.738036
2021-02-24 01:00:00,48876.094286,48789.475031,48796.528618,48788.886048,48792.236580,48795.932875,48816.524695,48775.172992,48750.732946,48854.503379,48887.485771
2021-02-24 01:20:00,49140.160000,49103.519930,49119.751027,49068.362483,49139.953266,49120.876832,49217.139154,49088.595725,49089.410952,49129.173259,49137.171587


In [14]:
# Dataframe contenant les amounts cumulés des 11 exchanges
amount_df.head()

,bfly,bfnx,bnus,btrx,cbse,gmni,itbi,krkn,lmax,okcn,stmp
timestamp,,,,,,,,,,,
2021-02-24 00:00:00,0.720115,237.912626,43.788192,17.355784,852.011329,77.049876,9.326294,128.250787,287.79,27.1232,136.437677
2021-02-24 00:20:00,0.475803,310.766469,60.480785,19.298085,745.545499,75.825521,14.332177,261.265215,312.70,8.1588,112.130796
2021-02-24 00:40:00,0.761515,78.035145,23.585851,5.813368,371.410017,30.804447,3.642059,94.685192,94.55,16.0403,65.785094
2021-02-24 01:00:00,0.210000,137.361093,49.333551,13.709603,514.145461,53.081503,8.284872,241.051335,145.43,13.1758,269.179638
2021-02-24 01:20:00,0.083427,250.684868,31.413929,14.988809,573.903286,46.109154,12.190996,137.723708,64.46,4.0278,143.656431


In [15]:
# Création d'un série contenant l'amount cumulé global pour chaque heure.
amount_global_serie = amount_df.sum(axis=1)

In [16]:
# Série contenant la somme des amounts cumulés des 11 exchanges
amount_global_serie.head()

timestamp
2021-02-24 00:00:00    1817.765881
2021-02-24 00:20:00    1920.979149
2021-02-24 00:40:00     785.112988
2021-02-24 01:00:00    1444.962856
2021-02-24 01:20:00    1279.242409
Freq: 20T, dtype: float64

In [17]:
# Créations de la colonne vwap global
vwap_df['vwap global'] = 0
for nom in liste_noms:
    # Dans la boucle for, calcul par itération du numérateur du vwap global
    vwap_df['vwap global'] += vwap_df[nom]* amount_df[nom]
    
# hors de la boucle for, division par le dénominateur
vwap_df['vwap global'] /= amount_global_serie
vwap_df.head()

,bfly,bfnx,bnus,btrx,cbse,gmni,itbi,krkn,lmax,okcn,stmp,vwap global
timestamp,,,,,,,,,,,,
2021-02-24 00:00:00,48249.222150,48258.479492,48332.214856,48362.278289,48395.779288,48634.707736,48320.808471,48167.026516,48457.997296,48094.876170,48472.607232,48380.630624
2021-02-24 00:20:00,47444.868821,47362.339719,47456.668198,47357.619664,47408.711379,47377.433422,47359.097461,47339.089475,47344.067631,47362.588674,47309.322432,47374.621185
2021-02-24 00:40:00,48129.147408,47976.235747,47945.813147,48037.306405,47989.239927,47835.837186,47993.418746,48054.618736,47900.208287,48018.952629,47930.738036,47974.002851
2021-02-24 01:00:00,48876.094286,48789.475031,48796.528618,48788.886048,48792.236580,48795.932875,48816.524695,48775.172992,48750.732946,48854.503379,48887.485771,48803.663863
2021-02-24 01:20:00,49140.160000,49103.519930,49119.751027,49068.362483,49139.953266,49120.876832,49217.139154,49088.595725,49089.410952,49129.173259,49137.171587,49123.104449


In [18]:
# Export du dataframe au format csv
vwap_df.to_csv('output_data/output_vwap_'+time_sample+'.csv', index=True)

# DEUXIEME PARTIE

Calcul des STD et OHLC

## STD

Pour les STD nous avons décidé de remplacer les écart-types manquants bien que, comme les OHLC, remplacer ces valeurs n'a pas beaucoup de sens puisqu'elles expliquent un écart entre des transactions qui n'ont pas eu lieu.
Néanmoins l'écart-type exprime aussi la volatilité de la monnaie, pour cela, nous avons arbitrairement choisi de combler ces NaN avec le std de la même tranche horaire de l'exchange BFNX, comme nous aurions pu choisir un autre exchange. La volatilité est une tendance générale qui s'applique à tout le marché en même temps.

In [19]:
# Création d'un nouveau dictionnaire destiné à accueillir les STD
dict_std = {}

# Cette boucle for resample avec la fonction STD en sortie
for k,v in dict_init.items():
    dict_std[k] = v.resample(time_sample).std()
#dict_std

In [20]:
# Cette boucle for élimine la colonne 'amount' après le resampling, seul l'écart-type du prix nous intéresse
for k,v in dict_std.items():
    dict_std[k] = v.drop(axis=1, columns='amount')
#dict_std

In [21]:
# Cette boucle renomme les colonnes "prix" en "std"
for df in dict_std.values():
    df.rename(columns={'price': 'std'}, inplace=True)

In [22]:
# Concatenation horizontal des 11 dataframes en un seul, chaque colonne est renommée par le nom de l'exchange
std_df = pd.concat(dict_std.values(), axis=1, keys=dict_std.keys())

In [23]:
# Observation de la présence de valeurs manquantes
std_df.head(10)

,bfly,bfnx,bnus,btrx,cbse,gmni,itbi,krkn,lmax,okcn,stmp
,std,std,std,std,std,std,std,std,std,std,std
timestamp,,,,,,,,,,,
2021-02-24 00:00:00,450.197677,482.080019,468.754422,422.827942,429.092781,445.244975,412.470053,333.107741,448.758336,370.249530,447.391077
2021-02-24 00:20:00,191.748369,224.779778,216.508257,231.518478,218.278502,222.569463,226.673750,203.941099,230.268118,220.148549,220.401055
2021-02-24 00:40:00,25.447744,246.680336,260.431515,239.052897,257.830463,220.378978,233.592070,258.131382,232.392890,242.059899,257.099285
2021-02-24 01:00:00,95.770542,149.926091,155.704647,149.110391,153.239825,150.318556,157.022096,155.069471,131.063828,145.903850,171.890244
2021-02-24 01:20:00,NaN,131.022889,135.052604,135.853630,139.904190,146.869934,134.182897,147.852963,150.884787,143.932025,144.214157
2021-02-24 01:40:00,158.966723,218.989160,176.014055,188.152620,189.764206,207.569033,173.916554,198.877298,166.163051,156.116060,200.862711
2021-02-24 02:00:00,135.097021,165.442916,181.761065,181.239432,189.621825,194.897941,175.862146,196.160117,189.246317,178.558184,196.450466
2021-02-24 02:20:00,NaN,202.083709,196.228831,174.199924,182.578235,187.879112,177.857843,190.453874,185.793278,178.510458,168.413646


In [24]:
# Remplacement des valeurs manquantes par celles de l'exchange BFNX
std_df = std_df.bfill(axis=1)

In [25]:
# Présentation du dataframe final
std_df

,bfly,bfnx,bnus,btrx,cbse,gmni,itbi,krkn,lmax,okcn,stmp
,std,std,std,std,std,std,std,std,std,std,std
timestamp,,,,,,,,,,,
2021-02-24 00:00:00,450.197677,482.080019,468.754422,422.827942,429.092781,445.244975,412.470053,333.107741,448.758336,370.249530,447.391077
2021-02-24 00:20:00,191.748369,224.779778,216.508257,231.518478,218.278502,222.569463,226.673750,203.941099,230.268118,220.148549,220.401055
2021-02-24 00:40:00,25.447744,246.680336,260.431515,239.052897,257.830463,220.378978,233.592070,258.131382,232.392890,242.059899,257.099285
2021-02-24 01:00:00,95.770542,149.926091,155.704647,149.110391,153.239825,150.318556,157.022096,155.069471,131.063828,145.903850,171.890244
2021-02-24 01:20:00,131.022889,131.022889,135.052604,135.853630,139.904190,146.869934,134.182897,147.852963,150.884787,143.932025,144.214157
...,...,...,...,...,...,...,...,...,...,...,...
2021-02-24 22:20:00,164.762038,164.762038,185.388938,162.334312,186.789059,176.672608,198.134773,197.256891,178.128400,168.693463,197.818195
2021-02-24 22:40:00,101.990178,101.990178,90.134319,96.030932,97.322310,100.207583,89.543788,101.638958,102.177099,86.708861,91.330011


In [26]:
# Export du dataframe au format csv
std_df.to_csv('output_data/output_std_'+time_sample+'.csv', index=True)

## OHLC

Pour le OHLC, nous decidons de ne pas remplacer les valeurs manquantes, en effet, s'il n'y a pas de transaction dans une période donnée, il ne peut pas y avoir de prix d'ouverture, de fermeture, de maximum ou de minimum. Il faut prendre les valeurs manquantes comme une donnée à part entière.
De plus l'exchange qui en a le plus n'est en rien comparable aux autres,c'est BFLY, il n'a que 234 transactions en 24h, là ou les autres en ont en dizaines et centaines de milliers. Son poids est donc infime en comparaison aux autres.

In [27]:
# Création d'un nouveau dictionnaire destiné à accueillir les OHLC
dict_ohlc = dict_init

# Cette boucle for élimine la colonne 'amount' avant le resampling
for k,v in dict_ohlc.items():
    dict_ohlc[k] = v.drop(axis=1, columns='amount')
#dict_ohlc

In [28]:
# Cette boucle for resample avec la fonction OHLC en sortie
for k,v in dict_init.items():
    dict_ohlc[k] = v.resample(time_sample).ohlc()
#dict_ohlc

In [29]:
# Cette boucle renomme les colonnes "prix" en "ohlc"
for df in dict_ohlc.values():
    df.rename(columns={'price': 'ohlc'}, inplace=True)
#dict_ohlc

In [30]:
# Concatenation vertical des 11 dataframes en un seul pour une meilleur lisibilité
ohlc_df = pd.concat(dict_ohlc.values(), axis=0, keys=dict_ohlc.keys())

In [31]:
# Présentation du dataframe final
ohlc_df

ohlc                              
                              open      high       low     close
     timestamp                                                  
bfly 2021-02-24 00:00:00  48929.54  48929.54  47648.25  47817.29
     2021-02-24 00:20:00  47500.00  47741.14  47086.89  47741.14
     2021-02-24 00:40:00  48128.38  48191.14  48128.38  48191.14
     2021-02-24 01:00:00  48934.14  48934.14  48798.70  48798.70
     2021-02-24 01:20:00  49140.16  49140.16  49140.16  49140.16
...                            ...       ...       ...       ...
stmp 2021-02-24 22:20:00  48344.02  48995.20  48294.90  48708.87
     2021-02-24 22:40:00  48707.54  48938.49  48522.96  48843.17
     2021-02-24 23:00:00  48831.81  49558.02  48798.61  49176.76
     2021-02-24 23:20:00  49170.35  49789.41  49170.35  49635.01
     2021-02-24 23:40:00  49608.66  49774.36  49493.89  49754.00

[792 rows x 4 columns]

In [32]:
# Export du dataframe au format csv
ohlc_df.to_csv('output_data/output_ohlc_'+time_sample+'.csv', index=True)